In [20]:
import utils 
import importlib

importlib.reload(utils)

<module 'utils' from 'c:\\Users\\halewijnvanden\\OneDrive - Delft University of Technology\\Documents\\PlatformIO\\Projects\\wheather_station\\streamlit_app\\utils.py'>

In [21]:
utils.get_google_sheet_df()

,end_device_ids_device_id,end_device_ids_application_ids_application_id,end_device_ids_dev_eui,end_device_ids_join_eui,end_device_ids_dev_addr,correlation_ids_0,received_at,uplink_message_session_key_id,uplink_message_f_port,uplink_message_f_cnt,...,uplink_message_version_ids_firmware_version,uplink_message_version_ids_band_id,uplink_message_network_ids_net_id,uplink_message_network_ids_ns_id,uplink_message_network_ids_tenant_id,uplink_message_network_ids_cluster_id,uplink_message_network_ids_cluster_address,uplink_message_last_battery_percentage_f_cnt,uplink_message_last_battery_percentage_value,uplink_message_last_battery_percentage_received_at
0,cubecell,wheather-station,70B3D57ED0074825,70B3D57ED0074824,260B7F34,gs:uplink:01KBZZJEAVC4A2F340S2099Y4Y,2025-12-08T21:58:34.794406371Z,AZr/jtEkw5xytMc/QqpqqQ==,2,116,...,1,EU_863_870,13,EC656E0000000181,ttn,eu1,eu1.cloud.thethings.network,99,NaN,2025-12-08T21:41:25.274050459Z
1,cubecell,wheather-station,70B3D57ED0074825,70B3D57ED0074824,260B7F34,gs:uplink:01KBZZM9381BZ0KWFY461HDY1T,2025-12-08T21:59:34.964237737Z,AZr/jtEkw5xytMc/QqpqqQ==,2,117,...,1,EU_863_870,13,EC656E0000000181,ttn,eu1,eu1.cloud.thethings.network,99,NaN,2025-12-08T21:41:25.274050459Z
2,cubecell,wheather-station,70B3D57ED0074825,70B3D57ED0074824,260B7F34,gs:uplink:01KBZZP41N5AEAG4Q83YNDW2YT,2025-12-08T22:00:35.328881046Z,AZr/jtEkw5xytMc/QqpqqQ==,2,118,...,1,EU_863_870,13,EC656E0000000181,ttn,eu1,eu1.cloud.thethings.network,99,NaN,2025-12-08T21:41:25.274050459Z
3,cubecell,wheather-station,70B3D57ED0074825,70B3D57ED0074824,260B7F34,gs:uplink:01KBZZQZAHZMPCSCJYMQ0JJNKX,2025-12-08T22:01:36.030115798Z,AZr/jtEkw5xytMc/QqpqqQ==,2,119,...,1,EU_863_870,13,EC656E0000000181,ttn,eu1,eu1.cloud.thethings.network,99,NaN,2025-12-08T21:41:25.274050459Z
4,cubecell,wheather-station,70B3D57ED0074825,70B3D57ED0074824,260B7F34,gs:uplink:01KBZZSTTNB2XQD2V7TABNTYSV,2025-12-08T22:02:36.961741002Z,AZr/jtEkw5xytMc/QqpqqQ==,2,120,...,1,EU_863_870,13,EC656E0000000181,ttn,eu1,eu1.cloud.thethings.network,99,NaN,2025-12-08T21:41:25.274050459Z
5,cubecell,wheather-station,70B3D57ED0074825,70B3D57ED0074824,260B7F34,gs:uplink:01KBZZVNPKKA367WHT3TY6K07A,2025-12-08T22:03:37.252423307Z,AZr/jtEkw5xytMc/QqpqqQ==,2,121,...,1,EU_863_870,13,EC656E0000000181,ttn,eu1,eu1.cloud.thethings.network,99,NaN,2025-12-08T21:41:25.274050459Z
6,cubecell,wheather-station,70B3D57ED0074825,70B3D57ED0074824,260B7F34,gs:uplink:01KBZZXGPP5JXEBRXDB5S8SFTD,2025-12-08T22:04:37.665993867Z,AZr/jtEkw5xytMc/QqpqqQ==,2,122,...,1,EU_863_870,13,EC656E0000000181,ttn,eu1,eu1.cloud.thethings.network,99,NaN,2025-12-08T21:41:25.274050459Z
7,cubecell,wheather-station,70B3D57ED0074825,70B3D57ED0074824,260B7F34,gs:uplink:01KBZZZBHPFQ4W8BKJXN3HJK94,2025-12-08T22:05:37.925754391Z,AZr/jtEkw5xytMc/QqpqqQ==,2,123,...,1,EU_863_870,13,EC656E0000000181,ttn,eu1,eu1.cloud.thethings.network,99,NaN,2025-12-08T21:41:25.274050459Z
8,cubecell,wheather-station,70B3D57ED0074825,70B3D57ED0074824,260B7F34,gs:uplink:01KC0016EVNA90N92V17WMHK47,2025-12-08T22:06:38.247267475Z,AZr/jtEkw5xytMc/QqpqqQ==,2,124,...,1,EU_863_870,13,EC656E0000000181,ttn,eu1,eu1.cloud.thethings.network,99,NaN,2025-12-08T21:41:25.274050459Z
9,cubecell,wheather-station,70B3D57ED0074825,70B3D57ED0074824,260B7F34,gs:uplink:01KC0031T60NEPHYE7ANRCTNYZ,2025-12-08T22:07:39.030269690Z,AZr/jtEkw5xytMc/QqpqqQ==,2,125,...,1,EU_863_870,13,EC656E0000000181,ttn,eu1,eu1.cloud.thethings.network,99,NaN,2025-12-08T21:41:25.274050459Z
